In [2]:
pip install torch



  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/74/07/edce54779f5c3fe8ab8390eafad3d7c8190fce68f922a254ea77f4a94a99/torch-2.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/5e/5d/97afbafd9d584ff1b45fcb354a479a3609bd97f912f8f1f6c563cb1fae21/filelock-3.12.4-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB 326.8 kB/s eta 0:00:18
      --------------------------------------- 0.1/5.7 MB 581.0 kB/s eta 0:00:10
     - -------------------------------------- 0.2/5.7 MB 817.0 kB/s eta 0:00:07
     - -------------------------------------- 0.2/5.7 MB 841.6 kB/s eta 0:00:07
     - -------------------------------------- 0.3/5.7 MB 980

In [6]:
pip install tqdm


  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.6 kB 131.3 kB/s eta 0:00:01
     ------------- ------------------------ 20.5/57.6 kB 131.3 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.6 kB 178.6 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.6 kB 201.8 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 201.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   -------------------- ------------------- 41.0/78.3 kB 991.0 kB/s eta 0:00:01
   ------------------------------------ --- 71.7/78.3 kB 787.7 kB/s eta 0:00:01
   ---------------------------

In [7]:
pip install torchnet

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.4 MB 393.8 kB/s eta 0:00:04
     -- ------------------------------------- 0.1/1.4 MB 573.4 kB/s eta 0:00:03
     --- ------------------------------------ 0.1/1.4 MB 654.9 kB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.4 MB 841.6 kB/s eta 0:00:02
     --------- ------------------------------ 0.3/1.4 MB 1.1 MB/s eta 0:00:01
     --------------- ------------------------ 0.5/1.4 MB 1.7 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.4 MB 1.9 MB/s eta 0:00:01
     ------------------------------- -------- 1.1/1.4 MB 2.7 MB/s eta 0:00:01
     ----------------------------------- ---- 1.3/1.4 MB 2.7 MB/s eta 0:00:01
     ---------------------------------------- 

In [3]:
import torch
from torch import nn


In [4]:
NUM_EPOCHS = 100
BATCH_SIZE = 256
CHANNEL_SIZE = 4
USE_CUDA = False

In [9]:
class RadioTransformerNetwork(nn.Module):
    def __init__(self, in_channels, compressed_dim):
        super(RadioTransformerNetwork, self).__init__()

        self.in_channels = in_channels

        self.encoder = nn.Sequential(
            nn.Linear(in_channels, in_channels),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels, compressed_dim),
        )

        self.decoder = nn.Sequential(
            nn.Linear(compressed_dim, compressed_dim),
            nn.ReLU(inplace=True),
            nn.Linear(compressed_dim, in_channels)
        )

    def decode_signal(self, x):
        return self.decoder(x)

    def forward(self, x):
        x = self.encoder(x)

        # Normalization.
        x = (self.in_channels ** 2) * (x / x.norm(dim=-1)[:, None])

        # 7dBW to SNR.
        training_signal_noise_ratio = 5.01187

        # bit / channel_use
        communication_rate = 1

        # Simulated Gaussian noise.
        noise = Variable(torch.randn(*x.size()) / ((2 * communication_rate * training_signal_noise_ratio) ** 0.5))
        if USE_CUDA: noise = noise.cuda()
        x += noise

        x = self.decoder(x)

        return x


if __name__ == "__main__":
    from tqdm import tqdm
    from torchnet.engine import Engine
    from torch.autograd import Variable
    from torch.optim import Adam
    import torchnet as tnt
    import math

    model = RadioTransformerNetwork(CHANNEL_SIZE, compressed_dim=int(math.log2(CHANNEL_SIZE)))
    if USE_CUDA: model = model.cuda()

    train_labels = (torch.rand(10000) * CHANNEL_SIZE).long()
    train_data = torch.sparse.torch.eye(CHANNEL_SIZE).index_select(dim=0, index=train_labels)

    test_labels = (torch.rand(1500) * CHANNEL_SIZE).long()
    test_data = torch.sparse.torch.eye(CHANNEL_SIZE).index_select(dim=0, index=test_labels)

    optimizer = Adam(model.parameters())

    engine = Engine()
    meter_loss = tnt.meter.AverageValueMeter()
    meter_accuracy = tnt.meter.ClassErrorMeter(accuracy=True)
    confusion_meter = tnt.meter.ConfusionMeter(CHANNEL_SIZE, normalized=True)

    loss_fn = nn.CrossEntropyLoss()


    def get_iterator(mode):
        data = train_data if mode else test_data
        labels = train_labels if mode else test_labels
        tensor_dataset = tnt.dataset.TensorDataset([data, labels])

        return tensor_dataset.parallel(batch_size=BATCH_SIZE, num_workers=4, shuffle=mode)


    def processor(sample):
        data, labels, training = sample

        data = Variable(data)
        labels = Variable(labels)

        if USE_CUDA:
            data = data.cuda()
            labels = labels.cuda()

        outputs = model(data)

        loss = loss_fn(outputs, labels)

        return loss, outputs


    def reset_meters():
        meter_accuracy.reset()
        meter_loss.reset()
        confusion_meter.reset()


    def on_sample(state):
        state['sample'].append(state['train'])


    def on_forward(state):
        meter_accuracy.add(state['output'].data, torch.LongTensor(state['sample'][1]))
        confusion_meter.add(state['output'].data, torch.LongTensor(state['sample'][1]))
        meter_loss.add(state['loss'].item())


    def on_start_epoch(state):
        reset_meters()
        state['iterator'] = tqdm(state['iterator'])


    def on_end_epoch(state):
        print('[Epoch %d] Training Loss: %.4f (Accuracy: %.2f%%)' % (
            state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

        reset_meters()

        engine.test(processor, get_iterator(False))

        print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
            state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))


    engine.hooks['on_sample'] = on_sample
    engine.hooks['on_forward'] = on_forward
    engine.hooks['on_start_epoch'] = on_start_epoch
    engine.hooks['on_end_epoch'] = on_end_epoch

    engine.train(processor, get_iterator(True), maxepoch=NUM_EPOCHS, optimizer=optimizer)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:10<00:00,  3.87it/s]


[Epoch 1] Training Loss: 1.6497 (Accuracy: 21.57%)
[Epoch 1] Testing Loss: 1.5513 (Accuracy: 24.47%)


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:09<00:00,  4.21it/s]


[Epoch 2] Training Loss: 1.4754 (Accuracy: 25.49%)
[Epoch 2] Testing Loss: 1.4281 (Accuracy: 24.47%)


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:14<00:00,  2.83it/s]


...

[Epoch 98] Training Loss: 0.3614 (Accuracy: 75.56%)
[Epoch 98] Testing Loss: 0.3679 (Accuracy: 74.53%)


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:08<00:00,  4.67it/s]


[Epoch 99] Training Loss: 0.3654 (Accuracy: 75.56%)
[Epoch 99] Testing Loss: 0.3674 (Accuracy: 74.53%)


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.27it/s]


[Epoch 100] Training Loss: 0.3615 (Accuracy: 75.56%)
[Epoch 100] Testing Loss: 0.3669 (Accuracy: 74.53%)
